# Idea

Find times of the quarter with big movements in price and watch the drift of the asset for several periods after. Trade based on that (either hypothesizing momentum or reversal).

Example signal: long or short assets for days or a couple of weeks right after earnings according to size and direction of 1-day, post-event returns (similar to "post-earnings announcement drift").